In [11]:
try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup  
import urllib2

year = 2013
# results of municipal elections 
# the last number in the ID goes from 01 till 55
start_url = 'https://www.kmdvalg.dk/kv/{}/K84982101001.htm'.format(year)

def parse_results (url):
    contents = urllib2.urlopen(start_url).read()
    parsed = BeautifulSoup(contents)
    votes = {}
    for i, row in enumerate(parsed.find_all('div', {'class': 'row table-like-row'})):

        if i > 0:
            for j, cell in enumerate(row.find_all('div', {'class': 'table-like-cell'})):
                if j == 0:
                    party = cell.next.next.next.next
                    votes[party] = {}
                if j == 1:
                    votes[party]['total'] = int(cell.next.replace('.', ''))
                if j == 2:
                    votes[party]['delta'] = int(cell.next.replace('.', ''))

    return votes

id_votes = {}
for i in range(1, 56):
    
    url = start_url[:41] + "%02d" % (i,) + '.htm'
    try:
        id_votes[url] = parse_results(url)
    except (e):
        print e

In [12]:
from collections import defaultdict
import json 

mapping = {
    'østerbro': [1, 2, 3, 5, 6],
    'amager_vest': [7, 8, 9, 10],
    'indre_by': [11, 12, 13, 14],
    'amager_øst': [16, 17, 18, 19],
    'nørrebro': [20, 21, 22, 23, 24, 25, 26],
    'bispebjerg': [27, 28, 29, 30],
    'bronshøj-husum': [30, 31, 32, 33, 34, 35, 36],
    'vanløse': [39, 40, 41],
    'valby': [42, 43, 44, 46, 37, 48, 55],
    'vesterbro-kongens_enghave':  [48, 50, 51, 52, 53, 54],
}

mapping_ID = {
    2: [1, 2, 3, 5, 6],
    10: [7, 8, 9, 10],
    1: [11, 12, 13, 14],
    9: [16, 17, 18, 19],
    3: [20, 21, 22, 23, 24, 25, 26],
    8: [27, 28, 29, 30],
    7: [30, 31, 32, 33, 34, 35, 36],
    6: [39, 40, 41],
    5: [42, 43, 44, 46, 37, 48, 55],
    4:  [48, 50, 51, 52, 53, 54],
}
district_votings = defaultdict(list)

with open('district_ids.json') as f:
    district_ids = json.load(f)

for dis in mapping_ID:
    for i in mapping_ID[dis]:
        url = start_url[:41] + "%02d" % (i,) + '.htm'
        try:
            district_votings[dis].append(id_votes[url])
        except Exception as e:
            print e, dis.encode('utf-8')
            print url

with open('{}_district_results_ID.json'.format(year), 'w') as outfile:
    json.dump(district_votings, outfile)
